In [1]:
import pandas as pd

In [3]:
# load HDI data and get ISO A3 code
hdi_df = pd.read_json("../data/hdi_2021.json")
hdi_df["iso_a3_code"] = hdi_df["country"].str[:3]
display(hdi_df)

,country,index,indicator,year,value,iso_a3_code
0,AFG - Afghanistan,HDI - Human Development Index,hdi - Human Development Index (value),2021,0.478,AFG
1,AGO - Angola,HDI - Human Development Index,hdi - Human Development Index (value),2021,0.586,AGO
2,ALB - Albania,HDI - Human Development Index,hdi - Human Development Index (value),2021,0.796,ALB
3,AND - Andorra,HDI - Human Development Index,hdi - Human Development Index (value),2021,0.858,AND
4,ARE - United Arab Emirates,HDI - Human Development Index,hdi - Human Development Index (value),2021,0.911,ARE
...,...,...,...,...,...,...
197,ZZG.ECA - Europe And Central Asia,HDI - Human Development Index,hdi - Human Development Index (value),2021,0.796,ZZG
198,ZZH.LAC - Latin America And The Caribbean,HDI - Human Development Index,hdi - Human Development Index (value),2021,0.754,ZZH
199,ZZI.SA - South Asia,HDI - Human Development Index,hdi - Human Development Index (value),2021,0.632,ZZI
200,ZZJ.SSA - Sub-Saharan Africa,HDI - Human Development Index,hdi - Human Development Index (value),2021,0.547,ZZJ
